In [1]:
import geopandas as gpd 
import geemap
import rasterio
import ee 

In [3]:
ee.Authenticate()
ee.Initialize()

# Manejo de archivos vectoriales

In [14]:
edom= gpd.read_file(r'C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex.shp')

In [16]:
# Simplificar las geometrías con una tolerancia deseada. Ajusta este valor según sea necesario.
# La tolerancia estará en las mismas unidades que el sistema de coordenadas del shapefile.
gdf_simplified = edom.simplify(tolerance=1.0)

# Guardar el shapefile simplificado
gdf_simplified.to_file(r'C:\Users\egonz\Documents\Curso\Variables\Edomex\prueba.shp')

In [17]:
import os

# Crea un mapa interactivo
Map = geemap.Map()

# Ruta al archivo shapefile
shp_file_path = r'C:\Users\egonz\Documents\Curso\Variables\Edomex\prueba.shp'

# Comprueba si el archivo shapefile existe
if not os.path.exists(shp_file_path):
    raise FileNotFoundError("El archivo shapefile no se encontró.")

# Convierte el shapefile a un Earth Engine FeatureCollection
ee_object = geemap.shp_to_ee(shp_file_path)

# Añade la capa al mapa
Map.addLayer(ee_object, {}, 'Nombre de la capa')

# Muestra el mapa
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Manejo de archivos raster

# Temperatura

In [5]:
# Recortar la imagen para evitar trabajar con bits innecesarios

mask= r"F:\Servicio INIFAP\MaizEdo_Actual\Insumos\Edomex.shp"
temp= r"C:\Users\egonz\Documents\Curso\Variables\Temp.tif"
out= r"C:\Users\egonz\Documents\Curso\Variables\TempClip.tif"

#tempClip= geemap.clip_image(temp,mask,out)

In [16]:
import numpy as np

In [27]:
# Carga la imagen raster desde tu disco duro
image = ee.Image(r'C:\Users\egonz\Documents\Curso\Variables\TempClip.tif')

# Define la tabla de reclasificación
reclassification = [
    [10,12,20],
    [12,15,70],
    [15, 22, 100],
    [22, 28, 85],
    [28, 32, 70],
    [32, 40, 20],
    [-np.inf, 10, 10],
    [40, np.inf,10]
]

In [24]:
def reclassify(image, reclassification):
    """
    Reclassifies the input image based on the given reclassification rules.

    Args:
        image (ee.Image): The input image to reclassify.
        reclassification (dict): A dictionary of reclassification rules.

    Returns:
        ee.Image: The reclassified image.
    """
    def assign_values(value):
        """
        Assigns new values to the input image based on the given reclassification rules.

        Args:
            value (ee.Number): The input pixel value.

        Returns:
            ee.Number: The new pixel value.
        """
        for old_value, new_value in reclassification.items():
            if value.eq(old_value):
                return ee.Number(new_value)
        return None

    # Applies the custom function to each pixel in the image
    reclassified = image.expression(
        'b(where(isnull(values), 0, assign_values(values)))',
        {'values': image,
         'assign_values': ee.Function.apply(
             ee.Algorithms.If,
             [ee.Number(assign_values(ee.Number('value'))).eq(None),
              0,
              ee.Number(assign_values(ee.Number('value')))]
         )}
    ).float()

    return reclassified

In [ ]:
def reclassify(image, reclassification):
    """
    Reclassifies the input image based on the given reclassification rules.

    Args:
        image (ee.Image): The input image to reclassify.
        reclassification (dict): A dictionary of reclassification rules.

    Returns:
        ee.Image: The reclassified image.
    """
    def assign_values(value):
        """
        Assigns new values to the input image based on the given reclassification rules.

        Args:
            value (ee.Number): The input pixel value.

        Returns:
            ee.Number: The new pixel value.
        """
        for old_value, new_value in reclassification.items():
            if value.eq(old_value):
                return ee.Number(new_value)
        return None

    # Applies the custom function to each pixel in the image
    reclassified = image.expression(
        'b(where(isnull(values), 0, assign_values(values)))',
        {'values': image,
         'assign_values': ee.Function.apply(
             ee.Algorithms.If,
             [ee.Number(assign_values(ee.Number('value'))).eq(None),
              0,
              ee.Number(assign_values(ee.Number('value')))]
         )}
    ).float()

    return reclassified

In [ ]:
import rasterio
import numpy as np

# Abre el archivo raster original
with rasterio.open('path_to_your_raster.tif') as src:
    raster_data = src.read(1)  # Lee la primera banda
    profile = src.profile

# Define tus reglas de reclasificación, por ejemplo:
# Si el valor de píxel está entre 0 y 100, se reclasifica a 1
# Si el valor de píxel está entre 101 y 200, se reclasifica a 2
# Si el valor de píxel es mayor que 200, se reclasifica a 3
reclassified_data = np.where((raster_data > 0) & (raster_data <= 100), 1, raster_data)
reclassified_data = np.where((raster_data > 100) & (raster_data <= 200), 2, reclassified_data)
reclassified_data = np.where(raster_data > 200, 3, reclassified_data)

# Guarda el raster reclasificado
with rasterio.open('path_to_reclassified_raster.tif', 'w', **profile) as dst:
    dst.write(reclassified_data, 1)
